### Importar librerías cirq y qsimcirq

In [10]:
try:
    import cirq
except ImportError:
    !pip install cirq --quiet
    import cirq

try:
    import qsimcirq
except ImportError:
    !pip install qsimcirq --quiet
    import qsimcirq

### Cargarmos el profiler scalene, para medir los tiempos

In [11]:
%load_ext scalene

The scalene extension is already loaded. To reload it, use:
  %reload_ext scalene


### Código qsim, algoritmo de Schrödinger

In [16]:
#%%scalene [--profile-all]
# Importamos cirq y qsim
import cirq
import qsimcirq
import time
import numpy as np
from scalene import scalene_profiler

# Inicializamos las opciones para el hardware de simulación
# Option 1 (mode=0) or Option 2 (mode=1)
# Option 3 (number of GPUs = `num_gpus`)
gpu_options = qsimcirq.QSimOptions(cpu_threads=96, use_gpu=False, gpu_mode = 0, max_fused_gate_size=4)
qsim_simulator = qsimcirq.QSimSimulator(qsim_options=gpu_options)

#-------------------------------------------------------

#valores iniciales
num_qubits=14
initial_num_qubits=13
depth=2
initial_depth=1
repeticiones=20 #número de veces que se simula el circuito aleatorio

#cálculo del tiempo para n qubits y d profundidades
for n in range(initial_num_qubits,num_qubits):

  # Inicializamos los qubits 
  #q0, q1 = cirq.LineQubit.range(2)
  #circuit = cirq.Circuit(cirq.H(q0), cirq.CX(q0, q1))
  qubits = cirq.GridQubit.rect(n, 1)
  
  #qsim_elapsed = np.zeros(depth-initial_depth+1)

  for d in range(initial_depth,depth): 
  
    #simulo r veces
    for r in range(repeticiones):
      # Generamos un circuito aleatorio con los qubits definidos
      circuit = cirq.experiments.random_rotations_between_grid_interaction_layers_circuit(qubits=qubits, depth=d)
      
      # Mostrar el circuito simulado
      print("Circuito simulado:")
      print(circuit)
      
      #qsim_start = time.time()

      #línea del código de la que se ha estudiado el tiempo. Calcula la evolución del vector de estado hasta obtener las amplitudes de probabilidad
      qsim_results = qsim_simulator.compute_amplitudes(circuit, bitstrings=[0b00, 0b01])
      #qsim_elapsed[d-initial_depth+1] += time.time() - qsim_start
      




Circuito simulado:
(0, 0): ────PhX(0.25)^0.5───@───Y^0.5───────────
                            │
(1, 0): ────X^0.5───────────@───PhX(0.25)^0.5───

(2, 0): ────X^0.5───────────@───PhX(0.25)^0.5───
                            │
(3, 0): ────PhX(0.25)^0.5───@───X^0.5───────────

(4, 0): ────PhX(0.25)^0.5───@───Y^0.5───────────
                            │
(5, 0): ────Y^0.5───────────@───X^0.5───────────

(6, 0): ────PhX(0.25)^0.5───@───X^0.5───────────
                            │
(7, 0): ────Y^0.5───────────@───X^0.5───────────

(8, 0): ────X^0.5───────────@───PhX(0.25)^0.5───
                            │
(9, 0): ────PhX(0.25)^0.5───@───Y^0.5───────────

(10, 0): ───Y^0.5───────────@───PhX(0.25)^0.5───
                            │
(11, 0): ───PhX(0.25)^0.5───@───Y^0.5───────────

(12, 0): ───Y^0.5───────────────X^0.5───────────
Circuito simulado:
(0, 0): ────X^0.5───────────@───Y^0.5───────────
                            │
(1, 0): ────Y^0.5───────────@───PhX(0.25)^0.5───

(2, 0): ──

### Código qsimh, algoritmo híbrido Schrödinger-Feynman

In [18]:
#%%scalene [--profile-all]
# Importamos Cirq y qsim
import cirq
import qsimcirq
import numpy as np


options = {} #aquí se definirán las opciones para partir el circuito según el algoritmo de Feynan

#valores iniciales
num_qubits=24
initial_num_qubits=23
depth=31
initial_depth=30
repeticiones=1
k=2 #numero de subcircuitos
w_final=int(initial_num_qubits/k) #referencia del corte

options['k'] = range(0, w_final) #todos los qubits que están en la primera mitad
options['p'] = 0 #qubits 'prefijo', no utilizados
options['r'] = 0 #qubits 'sufijo', no utilizado
qsimh_results=np.zeros(w_final)

#cálculo del tiempo para n qubits y d profundidades
for n in range(initial_num_qubits,num_qubits):
  # Inicializamos los qubits 
  qubits = cirq.GridQubit.rect(n, 1)

  for d in range(initial_depth,depth): 
  
    #simulo r veces 
    for r in range(repeticiones):
        
      # generamos el circuito aleatorio
      circuit = cirq.experiments.random_rotations_between_grid_interaction_layers_circuit(qubits=qubits, depth=d)
      # Mostrar el circuito simulado
      print("Circuito simulado:")
      print(circuit)

      #como con este algoritmo el circuito se divide en dos, en este bucle se calcula cada uno de los subcircuitos w por separado
      for w in range(0, k):
            options['w'] = w
            qsimh_simulator = qsimcirq.QSimhSimulator(options)
            results = qsimh_simulator.compute_amplitudes(circuit, bitstrings=[0b00, 0b01, 0b10, 0b11])
    
 



Circuito simulado:
(0, 0): ────PhX(0.25)^0.5───@───Y^0.5───────────────X^0.5───────────PhX(0.25)^0.5───X^0.5───────────Y^0.5───────────PhX(0.25)^0.5───@───Y^0.5───────────────X^0.5───────────@───PhX(0.25)^0.5───────X^0.5───────────Y^0.5───────────X^0.5───────────Y^0.5───────────X^0.5───────────@───Y^0.5───────────────X^0.5───────────@───PhX(0.25)^0.5───────X^0.5───────────PhX(0.25)^0.5───Y^0.5───────────PhX(0.25)^0.5───X^0.5───────────@───Y^0.5───────────────X^0.5───────────@───PhX(0.25)^0.5───────Y^0.5───────────PhX(0.25)^0.5───Y^0.5───────────PhX(0.25)^0.5───X^0.5───────────
                            │                                                                                                       │                                       │                                                                                                       │                                       │                                                                                                   

In [ ]:
#si quisiéramos ver los resultados de cada subcircuito:
options['w'] = 0

qsimh_simulator = qsimcirq.QSimhSimulator(options)
results_0 = qsimh_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01, 0b10, 0b11])
print(results_0)

In [ ]:
options['w'] = 1

qsimh_simulator = qsimcirq.QSimhSimulator(options)
results_1 = qsimh_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01, 0b10, 0b11])
print(results_1)

In [ ]:
#finalmente, se suman los resultados para ambos subcircuitos y se tendría lo mismo que para el algoritmo de Schrödinger
results = [r0 + r1 for r0, r1 in zip(results_0, results_1)]
print("qsimh results:")
print(results)
